In [1]:
import io
import gym
env = gym.make("Taxi-v3").env #创建一个环境
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
env.reset() # 设置新环境，随机状态
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
state = env.encode(3, 1, 2, 0) # (taxi所在的行，列，乘客状态，是不是到达目的地)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.s = 328  # 选择state328来演示没有训练的agent的随机过程

epochs = 0 
penalties, reward = 0, 0

frames = [] 

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # 将每个帧进行动画处理
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 467
Penalties incurred: 143


In [5]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
#         print(type(frame['frame']))
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 467
State: 0
Action: 5
Reward: 20


In [6]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(env.action_space.n)

6


In [7]:
%%time
"""训练"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1 #学习率
gamma = 0.6 # 折扣因子
epsilon = 0.1 #探索率

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0 #给定参数
    done = False
    #使用Q-learning算法进行更新
    while not done:
        if random.uniform(0, 1) < epsilon:#行动策略采用epsilon-greedy
            action = env.action_space.sample() # 探索新空间
        else:
            action = np.argmax(q_table[state]) # 利用已经学习到的值

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])#评估策略采用贪婪策略
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10: #设置奖励和惩罚
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("训练结束.\n")

Episode: 100000
训练结束.

Wall time: 40.2 s


In [8]:
"""评估训练后agent的表现"""

total_epochs, total_penalties = 0, 0
episodes = 1000

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 1000 episodes:
Average timesteps per episode: 13.197
Average penalties per episode: 0.0


In [9]:
env.s = 488  # 设置一个展示环境

epochs = 0
penalties, reward = 0, 0

ai = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state]) #选择价值最大化的行动
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    ai.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 15
Penalties incurred: 0


In [11]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
#         print(type(frame['frame']))
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.4)
        
print_frames(ai)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 15
State: 0
Action: 5
Reward: 20
